In [1]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 27.5 MB/s 
     |████████████████████████████████| 7.6 MB 56.3 MB/s 
     |████████████████████████████████| 182 kB 78.8 MB/s 
--2022-12-20 14:36:50--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.02s   

2022-12-20 14:36:51 (237 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2022-12-20 14:36:51--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.gith

In [2]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [3]:
# GPU 사용
device = torch.device("cuda")

# Dataset 만들어서 불러오기 

In [ ]:
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
    # 중복제거
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=128,
        #pad_to_max_length=True,
        padding='max_length',
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = NSMCDataset("ratings_train.txt")
test_dataset = NSMCDataset("ratings_test.txt")

                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  4.915700e+04  49157.000000
mean   6.752945e+06      0.502695
std    2.937158e+06      0.499998
min    6.010000e+02      0.000000
25%    4.777143e+06      0.000000
50%    7.565415e+06      1.000000
75%    9.260204e+06      1.000000
max    1.027809e+07      1.000000


# Create Model

In [4]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
model.load_state_dict(torch.load("model.pt"))

In [ ]:
# 모델 레이어 보기
model

# Learn

In [ ]:
epochs = 5
batch_size = 16

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/9137 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 69.20447504520416 Accuracy: tensor(0.5150, device='cuda:0')
Batch Loss: 138.25060606002808 Accuracy: tensor(0.5259, device='cuda:0')
Batch Loss: 206.72395318746567 Accuracy: tensor(0.5379, device='cuda:0')
Batch Loss: 274.3099235892296 Accuracy: tensor(0.5480, device='cuda:0')
Batch Loss: 339.8695189356804 Accuracy: tensor(0.5604, device='cuda:0')
Batch Loss: 404.2079871892929 Accuracy: tensor(0.5718, device='cuda:0')
Batch Loss: 467.7221958935261 Accuracy: tensor(0.5835, device='cuda:0')
Batch Loss: 528.9588233828545 Accuracy: tensor(0.5946, device='cuda:0')
Batch Loss: 589.842657327652 Accuracy: tensor(0.6026, device='cuda:0')
Batch Loss: 650.7022848725319 Accuracy: tensor(0.6092, device='cuda:0')
Batch Loss: 710.8449636101723 Accuracy: tensor(0.6141, device='cuda:0')
Batch Loss: 770.1475127637386 Accuracy: tensor(0.6199, device='cuda:0')
Batch Loss: 825.6692996025085 Accuracy: tensor(0.6275, device='cuda:0')
Batch Loss: 882.9218274354935 Accuracy: tensor(0.6328, device='

  0%|          | 0/9137 [00:00<?, ?it/s]

Batch Loss: 39.73946642130613 Accuracy: tensor(0.8119, device='cuda:0')
Batch Loss: 78.82442361861467 Accuracy: tensor(0.8225, device='cuda:0')
Batch Loss: 118.9460252597928 Accuracy: tensor(0.8177, device='cuda:0')
Batch Loss: 159.16261402517557 Accuracy: tensor(0.8202, device='cuda:0')
Batch Loss: 196.395938269794 Accuracy: tensor(0.8224, device='cuda:0')
Batch Loss: 237.83962524682283 Accuracy: tensor(0.8196, device='cuda:0')
Batch Loss: 277.20836628228426 Accuracy: tensor(0.8200, device='cuda:0')
Batch Loss: 313.7875456660986 Accuracy: tensor(0.8227, device='cuda:0')
Batch Loss: 352.5112458765507 Accuracy: tensor(0.8226, device='cuda:0')
Batch Loss: 389.48197843134403 Accuracy: tensor(0.8234, device='cuda:0')
Batch Loss: 427.7793712615967 Accuracy: tensor(0.8239, device='cuda:0')
Batch Loss: 464.11392222344875 Accuracy: tensor(0.8244, device='cuda:0')
Batch Loss: 504.3683425039053 Accuracy: tensor(0.8232, device='cuda:0')
Batch Loss: 542.9689768254757 Accuracy: tensor(0.8234, devic

  0%|          | 0/9137 [00:00<?, ?it/s]

Batch Loss: 33.779012352228165 Accuracy: tensor(0.8456, device='cuda:0')
Batch Loss: 66.68339065462351 Accuracy: tensor(0.8534, device='cuda:0')
Batch Loss: 101.29717444628477 Accuracy: tensor(0.8485, device='cuda:0')
Batch Loss: 130.04890651628375 Accuracy: tensor(0.8553, device='cuda:0')
Batch Loss: 163.56981525942683 Accuracy: tensor(0.8551, device='cuda:0')


In [ ]:
losses, accuracies

테스트 데이터셋 정확도 확인하기

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "model.pt")